In [ ]:
from biasanalyzer.api import BIAS

# CREATE BIAS INSTANCE AND CONNECT TO DATABASE
bias = BIAS()
bias.set_config('../config.yaml')
bias.set_root_omop()

In [ ]:
import os

# DEFINE PATHS
source_dir = './yaml_tests/cohort_creation'
# print(source_dir)
destination_dir = os.path.abspath("./json_store")
# print(destination_dir)

In [ ]:
import json

# Walk through all subdirectories and files in the source directory
for root, dirs, files in os.walk(source_dir):
    for file in files:
        print(f'Start processing file: {file}')
        # Construct full file path
        source_file_path = os.path.join(root, file)
        print(f'source_file_path = {source_file_path}')
        base_filename = os.path.splitext(file)[0]  # removes .yaml

        # print(f'root = {root}')
        # Construct the corresponding destination path
        relative_path = os.path.relpath(root, source_dir)
        # print(f'relative_path: {relative_path}')
        destination_path = os.path.join(destination_dir, relative_path)
        # print(f'destination_path: {destination_path}')
        # Create the destination directory if it doesn't exist
        os.makedirs(destination_path, exist_ok=True)

        # create the cohort and get the concept stats
        # FUNCTION: def create_cohort(self, cohort_name, cohort_desc, query_or_yaml_file, created_by)
        cohort = bias.create_cohort(base_filename, f'{base_filename} description', source_file_path, 'system')
        print(f'cohort is empty = {cohort is None}')
        if cohort is None:
            continue

        concept_stats = cohort.get_concept_stats()
        print(f'concept_stats is empty = {concept_stats is None}')
        if concept_stats is None:
            continue
        # Save concept stats
        with open(os.path.join(destination_path, f"{base_filename}_concept_stats.json"), 'w') as afile:
            json.dump(concept_stats, afile)

        # GET DEMOGRAPHICS DATA FOR STUDY COHORT
        race_stats = cohort.get_stats('race')
        print(f'race_stats is empty = {race_stats is None}')
        if race_stats is None:
            continue
        # Save race stats
        with open(os.path.join(destination_path, f"{base_filename}_race_stats.json"), 'w') as afile:
            json.dump(race_stats, afile)

        age_distr = cohort.get_distributions('age')
        print(f'age_distr is empty = {age_distr is None}')
        if age_distr is None:
            continue
        # Save age distribution
        with open(os.path.join(destination_path, f"{base_filename}_age_distribution.json"), 'w') as afile:
            json.dump(age_distr, afile)

        gender_distr = cohort.get_distributions('gender')
        print(f'gender_distr is empty = {gender_distr is None}')
        if gender_distr is None:
            continue
        # Save gender distribution
        with open(os.path.join(destination_path, f"{base_filename}_gender_distribution.json"), 'w') as afile:
            json.dump(gender_distr, afile)


In [ ]:
# CLEAN-UP DATABASE
bias.cleanup()